In [9]:
from seagul.rl.ars.ars_pipe import ars
from seagul.nn import MLP
from common import *
from seagul.mesh import create_mesh, variation_dim, compute_meshdim
import scipy.optimize as opt
import torch
import matplotlib.pyplot as plt
import time
import copy
import gym
import torch

import xarray as xr
import numpy as np
import pandas as pd

import os

from mujoco_py.generated import const
from PIL import Image
torch.autograd.set_grad_enabled(False)

In [12]:
env_name = "HalfCheetah-v2"
post_name = 'identity'
seed = 1
save_dir = f'./images/{env_name}_{post_name}_{seed}/'


data = torch.load(f"./data_mcshdim4/{env_name}.xr")
policy_dict = data.policy_dict

data = torch.load(f"./data17/{env_name}.xr")
policy_dict['identity'] = data.policy_dict['identity']

In [5]:
run_seed = 461649783

policy = policy_dict[post_name][seed]

act_list = []
obs_list = []
rew_list = []

dtype = torch.float32
done = False
#   cur_step = 0

env = gym.make(env_name)

env.seed(int(run_seed))
obs = env.reset()

while not done:
    obs = torch.as_tensor(obs, dtype=dtype).detach()
    obs_list.append(obs.clone())

    act = policy(obs).detach()
    obs, rew, done, _ = env.step(act.numpy())


 
    env.render()
    time.sleep(.01)


    act_list.append(torch.as_tensor(act.clone()))
    rew_list.append(rew)

    #cur_step += 1

ep_length = len(rew_list)
ep_obs = torch.stack(obs_list)
ep_act = torch.stack(act_list)
ep_rew = torch.tensor(rew_list, dtype=dtype)
ep_rew = ep_rew.reshape(-1, 1)

torch.autograd.set_grad_enabled(True)

Creating window glfw


In [16]:
#run_seed = 1484414988
#run_seed = 1937481142
for post_name in ['identity', 'mdim_div']:
    save_dir = f'./images/{env_name}_{post_name}_{seed}/'

    #run_seed = 461649783 # cheetah fall
    policy = policy_dict[post_name][seed]

    torch.autograd.set_grad_enabled(False)

    act_list = []
    obs_list = []
    rew_list = []

    dtype = torch.float32
    done = False
    cur_step = 0

    env = gym.make(env_name)
    env.seed(run_seed)
    obs = env.reset()
    env.render(mode='rgb_array')

    env.unwrapped.viewer.cam.type = const.CAMERA_FIXED
    env.unwrapped.viewer.cam.fixedcamid+=1

    os.makedirs(save_dir, exist_ok=True)
    while not done:
        obs = torch.as_tensor(obs, dtype=dtype)
        obs_list.append(obs.clone())

        act = policy(obs).detach()
        obs, rew, done, _ = env.step(act.numpy())

        imarr = env.render(mode='rgb_array')
        im = Image.fromarray(imarr)
        im.save(save_dir + str(cur_step) + ".png")


        act_list.append(torch.as_tensor(act.clone()))
        rew_list.append(rew)

        cur_step += 1

    ep_length = len(rew_list)
    ep_obs = torch.stack(obs_list)
    ep_act = torch.stack(act_list)
    ep_rew = torch.tensor(rew_list, dtype=dtype)
    ep_rew = ep_rew.reshape(-1, 1)

    torch.autograd.set_grad_enabled(True)
    import cv2
    import os

    image_folder = save_dir
    video_name = save_dir + f'{env_name}_{post_name}_{seed}_{run_seed}.mp4'

    images = sorted([img for img in os.listdir(image_folder) if img.endswith(".png")], key = lambda x: int(x.split(".")[0]))
    frame = cv2.imread(os.path.join(image_folder, images[0]))
    height, width, layers = frame.shape

    #video = cv2.VideoWriter(video_name, 0, 1, (width,height))
    video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'MP4V'), 30, (width,height))

    for image in images:
        video.write(cv2.imread(os.path.join(image_folder, image)))

    cv2.destroyAllWindows()
    video.release()

In [13]:
for run_seed in np.random.randint(0,2**32-1,size=(1000,)):

    policy = policy_dict[post_name][seed]

    act_list = []
    obs_list = []
    rew_list = []

    dtype = torch.float32
    done = False
 #   cur_step = 0

    env = gym.make(env_name)

    env.seed(int(run_seed))
    obs = env.reset()

    for cur_step in range(100):
        obs = torch.as_tensor(obs, dtype=dtype).detach()
        obs_list.append(obs.clone())

        act = policy(obs).detach()
        obs, rew, done, _ = env.step(act.numpy())


        if not (-1 < obs[1] < 1):
            print(run_seed)
            #env.render()


        #env.render()
        #time.sleep(.01)


        act_list.append(torch.as_tensor(act.clone()))
        rew_list.append(rew)

        #cur_step += 1

    ep_length = len(rew_list)
    ep_obs = torch.stack(obs_list)
    ep_act = torch.stack(act_list)
    ep_rew = torch.tensor(rew_list, dtype=dtype)
    ep_rew = ep_rew.reshape(-1, 1)

    torch.autograd.set_grad_enabled(True)

KeyboardInterrupt: 

Exception ignored in: 'mujoco_py.cymj.PyMjData._set'
Traceback (most recent call last):
  File "mujoco_py/generated/wrappers.pxi", line 4727, in mujoco_py.cymj._wrap_mjtNum_1d
  File "/home/sgillen/miniconda3/envs/baselines2/lib/python3.6/site-packages/numpy/core/numeric.py", line 469, in asarray
    @set_module('numpy')
KeyboardInterrupt
